In [2]:
from PySide6.QtSql import QSqlDatabase
from mysql import connector

conn = connector.connect(    
    host='localhost',
    user='root',
    password='root')

cursor = conn.cursor()

# 检查并创建schema
cursor.execute("CREATE SCHEMA IF NOT EXISTS attendance_system")
cursor.execute("USE attendance_system")


In [3]:
# 该格子用于创建对应的表。包括学生表，教师表，课程表，课程学生表，考勤表

# 开始事务
conn.start_transaction()

try:
    # Create the schema
    create_schema = "CREATE SCHEMA IF NOT EXISTS attendance_system"
    cursor.execute(create_schema)

    # Switch to the new schema
    cursor.execute("USE attendance_system")

    # Create the teacher table
    create_teacher_table = """
    CREATE TABLE IF NOT EXISTS teacher (
        name VARCHAR(100) NOT NULL,
        username VARCHAR(100) NOT NULL PRIMARY KEY,
        password VARCHAR(100) NOT NULL
    )
    """
    cursor.execute(create_teacher_table)

    # Create the student table
    create_student_table = """
    CREATE TABLE IF NOT EXISTS student (
        id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        gender ENUM('男', '女') NULL,
        age CHAR(3) NULL,
        name VARCHAR(100) NULL
    )
    """
    cursor.execute(create_student_table)

    # Create the course table
    create_course_table = """
    CREATE TABLE IF NOT EXISTS course (
        course_name VARCHAR(100) NOT NULL PRIMARY KEY,
        teacher_username VARCHAR(100) NOT NULL,
        FOREIGN KEY (teacher_username) REFERENCES teacher(username) On DELETE CASCADE
    )
    """
    cursor.execute(create_course_table)

    # Create the course_student table
    create_course_student_table = """
    CREATE TABLE IF NOT EXISTS course_student (
        course_name VARCHAR(100) NOT NULL,
        student_id INT NOT NULL,
        PRIMARY KEY (course_name, student_id),
        FOREIGN KEY (course_name) REFERENCES course(course_name) ON DELETE CASCADE,
        FOREIGN KEY (student_id) REFERENCES student(id) ON DELETE CASCADE
    )
    """
    cursor.execute(create_course_student_table)

    # Create the attendance_info table
    create_attendance_info_table = """
    CREATE TABLE IF NOT EXISTS attendance_info (
        date DATE NOT NULL,
        stu_id INT NOT NULL,
        course_name VARCHAR(100) NOT NULL,
        is_attendance BOOLEAN NOT NULL,
        PRIMARY KEY (date, stu_id, course_name),
        FOREIGN KEY (stu_id) REFERENCES student(id) ON DELETE CASCADE,
        FOREIGN KEY (course_name) REFERENCES course(course_name) ON DELETE CASCADE
    )
    """
    cursor.execute(create_attendance_info_table)

    # Commit the changes
    conn.commit()
except Exception as e:
    # 如果发生错误，回滚事务
    conn.rollback()
    print(f"An error occurred: {e}")

In [9]:
# Retrieve the list of tables in the schema
cursor.execute("""
SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'attendance_system'
""")
tables = cursor.fetchall()

# Delete the content of each table
try:
    try:
        conn.rollback()
        print("回滚了之前可能存在的事务")
    except:
        pass
    # Start transaction
    conn.start_transaction()

    # Disable foreign key checks
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0")

    for table in tables:
        table_name = table[0]
        cursor.execute(f"DELETE FROM {table_name}")
        print(f"All content deleted from table: {table_name}")

    # Enable foreign key checks
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1")

    # Commit transaction
    conn.commit()
except Exception as e:
    # Rollback transaction in case of error
    conn.rollback()
    print(f"An error occurred: {e}")


回滚了之前可能存在的事务
All content deleted from table: attendance_info
All content deleted from table: course
All content deleted from table: course_student
All content deleted from table: student
All content deleted from table: teacher


In [ ]:
# 随机插入数据库的数据

import random
import datetime

# Generate teacher data
teacher_names = ["Alice", "Bob", "Charlie", "David", "Eva", "Frank", "Grace", "Hannah", "Ivy", "Jack",
                 "Kathy", "Leo", "Mona", "Nina", "Oscar", "Paul", "Quincy", "Rachel", "Steve", "Tina"]
teachers = [(name, name, name) for name in teacher_names]

# Generate course data and ensure each teacher has at least 3 courses
courses = []
for i in range(len(teachers)):
    for j in range(3):  # Assign at least 3 courses to each teacher
        course_name = f"Course_{len(courses) + 1}"
        teacher_username = teachers[i][1]
        courses.append((course_name, teacher_username))

# Assign remaining courses randomly to teachers
for i in range(len(courses), 60):  # Total 60 courses
    course_name = f"Course_{i + 1}"
    teacher_username = random.choice(teachers)[1]
    courses.append((course_name, teacher_username))

# Generate student data with realistic names
first_names = ["John", "Jane", "Michael", "Emily", "Chris", "Anna", "Tom", "Sophia", "James", "Olivia"]
last_names = ["Smith", "Johnson", "Brown", "Taylor", "Anderson", "Thomas", "Jackson", "White", "Harris", "Martin"]
students = [(i + 1, random.choice(['男', '女']), str(random.randint(18, 25)),
             f"{random.choice(first_names)} {random.choice(last_names)}") for i in range(500)]

# Generate course_student data ensuring no duplicates
course_student = []
for course in courses:
    course_name = course[0]
    student_ids = random.sample(range(1, 501), random.randint(30, 40))
    for student_id in student_ids:
        course_student.append((course_name, student_id))

# Generate attendance data
start_date = datetime.date(2025, 3, 1)
end_date = datetime.date(2025, 3, 26)
date_range = [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1)]

attendance_data = []
try:
    # Start transaction
    conn.start_transaction()

    # Insert teacher data
    cursor.executemany("INSERT INTO teacher (name, username, password) VALUES (%s, %s, %s)", teachers)

    # Insert course data
    cursor.executemany("INSERT INTO course (course_name, teacher_username) VALUES (%s, %s)", courses)

    # Insert student data
    cursor.executemany("INSERT INTO student (id, gender, age, name) VALUES (%s, %s, %s, %s)", students)

    # Insert course_student data
    cursor.executemany("INSERT INTO course_student (course_name, student_id) VALUES (%s, %s)", course_student)

    # Generate and insert attendance data
    for date in date_range:
        selected_courses = random.sample(courses, 10)  # Select 10 courses per day
        for course in selected_courses:
            course_name = course[0]
            course_students = [cs[1] for cs in course_student if cs[0] == course_name]
            attendance_rate = random.uniform(0.9, 0.99)
            num_attending = int(attendance_rate * len(course_students))
            attending_students = random.sample(course_students, num_attending)
            for student_id in course_students:
                is_attendance = student_id in attending_students
                attendance_data.append((date, student_id, course_name, is_attendance))

    cursor.executemany("INSERT INTO attendance_info (date, stu_id, course_name, is_attendance) VALUES (%s, %s, %s, %s)", attendance_data)

    # Commit transaction
    conn.commit()
except Exception as e:
    # Rollback transaction in case of error
    conn.rollback()
    print(f"An error occurred: {e}")

In [10]:
cursor.execute("""SELECT name, id, is_attendance
FROM attendance_info AS a INNER JOIN student AS b
WHERE a.stu_id = b.id and course_name = 'course_10' and date = '2025-3-22';""")
res = cursor.fetchall()
res[0]

('Olivia Johnson', 28, 1)